In [1]:
# Cell 1: Importing required libraries
import os
import glob
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout
import sounddevice as sd
import scipy.io.wavfile as wav
import tempfile
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import ImageTk, Image
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import csv


In [2]:
# Cell 2: Feature extraction and dataset loading
def extract_features(file_path):
    audio, sr = librosa.load(file_path, res_type='kaiser_fast')
    features = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13).T, axis=0)
    return features

dataset_path = r"C:\Users\moham\Downloads\Emotion_Recognition_Project\Speech_Emotion_Recognition\TESS Toronto emotional speech set data"

data, labels = [], []
for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)
    for file_path in glob.glob(os.path.join(folder_path, '*.wav')):
        features = extract_features(file_path)
        data.append(features)
        labels.append(folder_name)

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

X_train = np.array(X_train)[:, np.newaxis, :]
X_test = np.array(X_test)[:, np.newaxis, :]


In [3]:
# Cell 3: Build and train the LSTM model
model = Sequential()
model.add(TimeDistributed(Dense(256, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


c:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.0641 - loss: 2.8460 - val_accuracy: 0.0768 - val_loss: 2.6476
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0660 - loss: 2.6992 - val_accuracy: 0.1286 - val_loss: 2.6201
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0710 - loss: 2.6723 - val_accuracy: 0.0679 - val_loss: 2.6147
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0859 - loss: 2.6444 - val_accuracy: 0.1750 - val_loss: 2.5936
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.0875 - loss: 2.6388 - val_accuracy: 0.2143 - val_loss: 2.5509
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1033 - loss: 2.6020 - val_accuracy: 0.2607 - val_loss: 2.4687
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.1192 - loss: 2.5454 - val_accuracy: 0.2446 - val_loss: 2.3857
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.1484 - loss: 2.4610 - val_accuracy: 0.2929 - v

In [4]:
# Cell 4: Prediction logic
def predict_emotion(audio_file):
    features = extract_features(audio_file)
    features = features[np.newaxis, np.newaxis, :]
    predicted_probabilities = model.predict(features)[0]
    predicted_index = np.argmax(predicted_probabilities)
    predicted_label = label_encoder.classes_[predicted_index]
    
    emotion_mapping = {
        'YAF_angry': 'ANGRY', 'YAF_disgust': 'DISGUST', 'YAF_fear': 'FEAR',
        'YAF_happy': 'HAPPY', 'YAF_neutral': 'NEUTRAL', 'YAF_pleasant_surprised': 'SURPRISED', 'YAF_sad': 'SAD',
        'OAF_angry': 'ANGRY', 'OAF_disgust': 'DISGUST', 'OAF_Fear': 'FEAR',
        'OAF_happy': 'HAPPY', 'OAF_neutral': 'NEUTRAL', 'OAF_Pleasant_surprised': 'SURPRISED', 'OAF_Sad': 'SAD',
    }
    
    readable_emotion = emotion_mapping.get(predicted_label, "UNKNOWN")
    emotion_labels = ['HAPPY', 'SAD', 'ANGRY', 'SURPRISED', 'NEUTRAL', 'FEAR', 'DISGUST']
    probabilities = dict(zip(emotion_labels, [0]*len(emotion_labels)))
    if readable_emotion in probabilities:
        probabilities[readable_emotion] = 1.0  # set detected emotion prob to 1.0

    return readable_emotion, probabilities


In [5]:
# Cell 5: GUI class definition
class EmotionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Speech Emotion Recognition")
        self.root.geometry("1000x600")
        self.dark_mode = False

        self.base_emoji_path = r"C:\\Users\\moham\\Downloads\\Emotion_Recognition_Project\\Speech_Emotion_Recognition\\Emotion Emojis\\Emoji"
        self.logo_path = r"C:\\Users\\moham\\Downloads\\Emotion_Recognition_Project\\Speech_Emotion_Recognition\\images\\logo.png"
        self.emotion_to_emoji = {
            "HAPPY": os.path.join(self.base_emoji_path, "happy.gif"),
            "SAD": os.path.join(self.base_emoji_path, "sad.gif"),
            "ANGRY": os.path.join(self.base_emoji_path, "angry.gif"),
            "SURPRISED": os.path.join(self.base_emoji_path, "surprised.gif"),
            "NEUTRAL": os.path.join(self.base_emoji_path, "neutral.gif"),
            "FEAR": os.path.join(self.base_emoji_path, "fear.gif"),
            "DISGUST": os.path.join(self.base_emoji_path, "disgust.gif")
        }

        self.emoji_label = None
        self.emoji_image = None
        self.prediction_history = []
        self.create_menu()
        self.show_home_page()

    def create_menu(self):
        self.menubar = tk.Menu(self.root)
        app_menu = tk.Menu(self.menubar, tearoff=0)
        app_menu.add_command(label="User Manual", command=self.show_user_manual)
        app_menu.add_command(label="Toggle Dark Mode", command=self.toggle_dark_mode)
        app_menu.add_command(label="About the App", command=self.show_about_page)
        self.menubar.add_cascade(label="Menu", menu=app_menu)
        self.root.config(menu=self.menubar)

    def clear_window(self):
        for widget in self.root.winfo_children():
            if not isinstance(widget, tk.Menu):
                widget.destroy()

    def apply_theme(self):
        bg = '#1e1e1e' if self.dark_mode else 'white'
        fg = 'white' if self.dark_mode else 'black'
        self.root.configure(bg=bg)
        return bg, fg

    def back_button(self, command):
        tk.Button(self.root, text="\u2190 Back", command=command, bg='lightgrey').place(x=10, y=10)

    def show_home_page(self):
        self.clear_window()
        bg, fg = self.apply_theme()
        left_frame = tk.Frame(self.root, bg=bg)
        left_frame.pack(side='left', fill='both', expand=True)
        right_frame = tk.Frame(self.root, bg=bg)
        right_frame.pack(side='right', fill='both', expand=True)

        tk.Label(left_frame, text="\U0001F3A7 Speech Emotion Recognition", font=('Helvetica bold', 18), bg=bg, fg=fg).pack(pady=40)
        tk.Button(left_frame, text="\U0001F399 Record Audio", command=self.record_audio_page, bg='orange').pack(pady=10)
        tk.Button(left_frame, text="\U0001F4C1 Upload Audio", command=self.upload_audio_page, bg='lightgreen').pack(pady=10)
        tk.Button(left_frame, text="\U0001F4DC Prediction History", command=self.show_history_page, bg='lightblue').pack(pady=10)
        tk.Button(left_frame, text="\u274C Exit", command=self.confirm_exit, bg='red', fg='white').pack(pady=10)

        if os.path.exists(self.logo_path):
            logo_image = Image.open(self.logo_path).resize((400, 550), Image.Resampling.LANCZOS)
            logo_tk = ImageTk.PhotoImage(logo_image)
            tk.Label(right_frame, image=logo_tk, bg=bg).pack()
            self.logo_img = logo_tk  # retain reference

    def confirm_exit(self):
        if messagebox.askokcancel("Exit", "Do you really want to exit?"):
            self.root.destroy()


In [28]:
class EmotionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Speech Emotion Recognition")
        self.root.geometry("1000x600")
        self.dark_mode = False

        self.base_emoji_path = r"C:\Users\moham\Downloads\Emotion_Recognition_Project\Speech_Emotion_Recognition\Emotion Emojis\Emoji"
        self.logo_path = r"C:\\Users\\moham\\Downloads\\Emotion_Recognition_Project\\Speech_Emotion_Recognition\\images\\logo.png"
        self.emotion_to_emoji = {
            "HAPPY": os.path.join(self.base_emoji_path, "happy.gif"),
            "SAD": os.path.join(self.base_emoji_path, "sad.gif"),
            "ANGRY": os.path.join(self.base_emoji_path, "angry.gif"),
            "SURPRISED": os.path.join(self.base_emoji_path, "surprised.gif"),
            "NEUTRAL": os.path.join(self.base_emoji_path, "neutral.gif"),
            "FEAR": os.path.join(self.base_emoji_path, "fear.gif"),
            "DISGUST": os.path.join(self.base_emoji_path, "disgust.gif")
        }

        self.emoji_label = None
        self.emoji_image = None
        self.prediction_history = []

        self.create_menu()
        self.show_home_page()

    def clear_window(self):
        for widget in self.root.winfo_children():
            if not isinstance(widget, tk.Menu):
                widget.destroy()

    def apply_theme(self):
        bg = '#1e1e1e' if self.dark_mode else 'white'
        fg = 'white' if self.dark_mode else 'black'
        self.root.configure(bg=bg)
        return bg, fg

    def back_button(self, command):
        tk.Button(self.root, text="\u2190 Back", command=command, bg='lightgrey').place(x=10, y=10)

    def create_menu(self):
        self.menubar = tk.Menu(self.root)
        app_menu = tk.Menu(self.menubar, tearoff=0)
        app_menu.add_command(label="User Manual", command=self.show_user_manual)
        app_menu.add_command(label="Toggle Dark Mode", command=self.toggle_dark_mode)
        app_menu.add_command(label="About the App", command=self.show_about_page)
        self.menubar.add_cascade(label="Menu", menu=app_menu)
        self.root.config(menu=self.menubar)

    def show_home_page(self):
        self.clear_window()
        bg, fg = self.apply_theme()

        left_frame = tk.Frame(self.root, bg=bg)
        left_frame.pack(side='left', fill='both', expand=True)
        right_frame = tk.Frame(self.root, bg=bg)
        right_frame.pack(side='right', fill='both', expand=True)

        tk.Label(left_frame, text="\U0001F3A7 Speech Emotion Recognition", font=('Helvetica bold', 18), bg=bg, fg=fg).pack(pady=40)
        tk.Button(left_frame, text="\U0001F399 Record Audio", command=self.record_audio_page, bg='orange').pack(pady=10)
        tk.Button(left_frame, text="\U0001F4C1 Upload Audio", command=self.upload_audio_page, bg='lightgreen').pack(pady=10)
        tk.Button(left_frame, text="\U0001F4DC Prediction History", command=self.show_history_page, bg='lightblue').pack(pady=10)
        tk.Button(left_frame, text="\u274C Exit", command=self.confirm_exit, bg='red', fg='white').pack(pady=10)

        try:
            if os.path.exists(self.logo_path):
                logo_image = Image.open(self.logo_path)
                logo_image = logo_image.resize((400, 550), Image.Resampling.LANCZOS)
                self.logo_tk = ImageTk.PhotoImage(logo_image)
                logo_label = tk.Label(right_frame, image=self.logo_tk, bg=bg)
                logo_label.image = self.logo_tk
                logo_label.pack()
        except Exception as e:
            print(f"Error loading logo: {e}")

    def confirm_exit(self):
        if messagebox.askokcancel("Exit", "Do you really want to exit?"):
            self.root.destroy()

    def toggle_dark_mode(self):
        self.dark_mode = not self.dark_mode
        self.show_home_page()

    def show_user_manual(self):
        self.clear_window()
        self.back_button(self.show_home_page)
        bg, fg = self.apply_theme()
        manual = (
            "📘 USER MANUAL\n\n"
            "1. 🎙 Click 'Record Audio' to use your microphone.\n"
            "2. 📂 Click 'Upload Audio' to select a .wav file.\n"
            "3. See predicted emotion, emoji and chart.\n"
            "4. Check 'Prediction History' for past results.\n"
            "5. Only '.wav' files are supported."
        )
        tk.Label(self.root, text=manual, justify="left", font=('Arial', 12), wraplength=550, bg=bg, fg=fg).pack(padx=20, pady=30)

    def show_about_page(self):
        self.clear_window()
        self.back_button(self.show_home_page)
        bg, fg = self.apply_theme()
        about = (
            "ℹ️ ABOUT THE APP\n\n"
            "📌 FEATURES:\n"
            "- Record/upload audio\n- Predict emotion\n- Emoji output\n- Probability chart\n- Export CSV\n\n"
            "📌 APPLICATIONS:\n"
            "- AI Assistants\n- Customer Service\n- Mental Health\n\n"
            "📌 CREDITS:\n- Dataset: TESS\n- Developed in Python"
        )
        tk.Label(self.root, text=about, justify="left", font=('Arial', 12), wraplength=550, bg=bg, fg=fg).pack(padx=20, pady=30)

    def show_history_page(self):
        self.clear_window()
        self.back_button(self.show_home_page)
        bg, fg = self.apply_theme()
        tk.Label(self.root, text="📖 Prediction History", font=('Helvetica bold', 16), bg=bg, fg=fg).pack(pady=20)
        if self.prediction_history:
            for i, (file_name, emotion) in enumerate(self.prediction_history[::-1], start=1):
                tk.Label(self.root, text=f"{i}. {file_name} — Emotion: {emotion}", bg=bg, fg=fg).pack(anchor='w', padx=20)
            tk.Button(self.root, text="💾 Export as CSV", command=self.export_csv, bg='orange').pack(pady=10)
        else:
            tk.Label(self.root, text="No history yet.", bg=bg, fg=fg).pack()

    def export_csv(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
        if file_path:
            with open(file_path, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["File Name", "Predicted Emotion"])
                writer.writerows(self.prediction_history)
            messagebox.showinfo("Success", "History exported!")

    def upload_audio_page(self):
        self.clear_window()
        self.back_button(self.show_home_page)
        bg, fg = self.apply_theme()
        def choose_file():
            file_path = filedialog.askopenfilename(filetypes=[("WAV Audio", "*.wav")])
            if file_path:
                self.display_result(file_path, os.path.basename(file_path))
        tk.Label(self.root, text="📂 Upload WAV File", font=('Helvetica bold', 16), bg=bg, fg=fg).pack(pady=40)
        tk.Button(self.root, text="Upload Audio", command=choose_file, bg='orange').pack(pady=10)

    def record_audio_page(self):
        self.clear_window()
        self.back_button(self.show_home_page)
        bg, fg = self.apply_theme()
        def record_and_predict():
            duration = 4
            fs = 44100
            tk.Label(self.root, text="Recording...", fg="red", bg=bg).pack()
            recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
            sd.wait()
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
                wav.write(temp_file.name, fs, recording)
                self.display_result(temp_file.name, "Recorded_Audio.wav")
        tk.Label(self.root, text="🎙 Record Your Voice", font=('Helvetica bold', 16), bg=bg, fg=fg).pack(pady=40)
        tk.Button(self.root, text="Start Recording", command=record_and_predict, bg='orange').pack(pady=10)

    def display_result(self, file_path, file_name):
        self.clear_window()
        self.back_button(self.show_home_page)
        bg, fg = self.apply_theme()
        predicted_emotion, probabilities = predict_emotion(file_path)
        self.prediction_history.append((file_name, predicted_emotion))
        tk.Label(self.root, text="Predicted Emotion:", font=('Helvetica bold', 16), bg=bg, fg=fg).pack(pady=10)
        tk.Label(self.root, text=predicted_emotion, font=('Helvetica bold', 20), fg='blue', bg=bg).pack()

        emoji_path = self.emotion_to_emoji.get(predicted_emotion.upper())
        if emoji_path and os.path.exists(emoji_path):
            self.emoji_image = tk.PhotoImage(file=emoji_path)
            self.emoji_label = tk.Label(self.root, image=self.emoji_image, bg=bg)
            self.emoji_label.image = self.emoji_image
            self.emoji_label.pack(pady=10)

        self.show_emotion_bar_chart(probabilities)

    def show_emotion_bar_chart(self, probabilities):
        fig, ax = plt.subplots(figsize=(6, 3), dpi=100)
        emotions, probs = list(probabilities.keys()), list(probabilities.values())
        ax.bar(emotions, probs, color='skyblue')
        ax.set_title('Emotion Probabilities')
        ax.set_ylim(0, 1)
        ax.set_ylabel('Probability')
        canvas = FigureCanvasTkAgg(fig, master=self.root)
        canvas.draw()
        canvas.get_tk_widget().pack(pady=10)

In [29]:
# Cell 7: Run the app
if __name__ == "__main__":
    root = tk.Tk()
    app = EmotionApp(root)
    root.mainloop()
